# Challenge 3 - Avanzado

## Visualización de grandes bases de datos

### Eyder Uriel Kinil Cervera 

Este challenge avanzado permitirá a los estudiantes aplicar PySpark para analizar grandes volúmenes de datos relacionados con un tema cercano a su investigación de tesis, integrando los conocimientos adquiridos en los challenges anteriores.

### Objetivo general

Realizar un análisis avanzado utilizando PySpark, relacionando el proyecto con su tema de tesis.

### Objetivo especifico

Utilizar APIs avanzadas como Spark Streaming o Spark ML para trabajar con una gran base de datos en tiempo real o construir un modelo predictivo complejo. Los estudiantes deberán crear una presentación de su análisis y presentarla al grupo.

In [35]:
#Ruta del csv
file_path = "../winequality-white.csv"

## Pyspark con regresión logistica

In [36]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import col

In [37]:
# Crear sesión de Spark
spark = SparkSession.builder.appName("WineQualityPredictor").getOrCreate()

# Cargar el conjunto de datos
data = spark.read.csv(file_path, header=True, inferSchema=True, sep=";")

In [38]:
# Mostrar información del dataset
data.printSchema()
data.show(5)

root
 |-- fixed acidity: double (nullable = true)
 |-- volatile acidity: double (nullable = true)
 |-- citric acid: double (nullable = true)
 |-- residual sugar: double (nullable = true)
 |-- chlorides: double (nullable = true)
 |-- free sulfur dioxide: double (nullable = true)
 |-- total sulfur dioxide: double (nullable = true)
 |-- density: double (nullable = true)
 |-- pH: double (nullable = true)
 |-- sulphates: double (nullable = true)
 |-- alcohol: double (nullable = true)
 |-- quality: integer (nullable = true)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|          7.0|          

In [39]:
# Seleccionar columnas de características (sin la columna de calidad)
feature_columns = [col for col in data.columns if col != 'quality']

# Usar VectorAssembler para combinar las columnas en un solo vector de características
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
assembled_data = assembler.transform(data)

# Crear columna 'label' que será 1 si la calidad es alta (>= 7), 0 si es baja
data_with_label = assembled_data.withColumn("label", (col("quality") >= 7).cast("double"))

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
train_data, test_data = data_with_label.randomSplit([0.8, 0.2], seed=1234)

# Crear el modelo de regresión logística
lr = LogisticRegression(featuresCol="features", labelCol="label")

# Entrenar el modelo
lr_model = lr.fit(train_data)

In [40]:
# Hacer predicciones sobre los datos de prueba
predictions = lr_model.transform(test_data)

In [41]:
# Evaluar el modelo usando la curva ROC
evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction")
roc = evaluator.evaluate(predictions)
print(f"ROC del modelo: {roc}")

ROC del modelo: 0.6220870902355491


## Pyspark con support vector machine

In [46]:
#Importando 
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import col

In [47]:
# Crear sesión de Spark
spark = SparkSession.builder.appName("SVMExample").getOrCreate()

# Cargar el conjunto de datos
data = spark.read.csv(file_path, header=True, inferSchema=True, sep=";")

In [49]:
# Seleccionar las columnas de características (sin la columna de calidad)
feature_columns = [col for col in data.columns if col != 'quality']

# Usar VectorAssembler para combinar las columnas en un solo vector de características
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
assembled_data = assembler.transform(data)

# Crear columna 'label' que será 1 si la calidad es alta (>= 7), 0 si es baja
data_with_label = assembled_data.withColumn("label", (col("quality") >= 7).cast("double"))

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
train_data, test_data = data_with_label.randomSplit([0.8, 0.2], seed=1234)

# Crear el modelo de SVM (LinearSVC)
svm = LinearSVC(featuresCol="features", labelCol="label")

# Entrenar el modelo
svm_model = svm.fit(train_data)

# Hacer predicciones sobre los datos de prueba
predictions = svm_model.transform(test_data)

In [48]:
# Evaluar el modelo usando AUC (Área bajo la curva ROC)
evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction")
auc = evaluator.evaluate(predictions)
print(f"AUC del modelo: {auc}")

AUC del modelo: 0.5


### Conclusiones 

En este ejercicio se emplea la base de datos del vino blanco para probar un modelo ML de regrsión logistica en un entorno se Spark, se observa una presión similar de 0.62 vs 0.62 del modelo Ramdon Forest con la base de datos de vino tinto. 

Sin embargo al realizar la comparativa del modelo de regresion lineal vs el modelo SVM para la prediccion de la calidad del vino, se observa un desempeño mejor en la regresión logistica de 0.62 vs 0.5.
